## TOPIC MODELING
####  Source of the following code :a super simplified implementation:  https://bennett-holiday.medium.com/a-step-by-step-guide-to-writing-an-lda-program-in-python-690aa99119ea
#### more eloborate one can be found here: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [38]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [61]:
#!pip install -U gensim
#!pip install -U nltk
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 8.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 5.3 MB/s eta 0:00:00


In [40]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

In [41]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ozgealacam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
davidson_sentence_file =  "../day1_annotation_interrateragreement/labeled_data.csv"
davidson_df = pd.read_csv(davidson_sentence_file) 
davidson_text= davidson_df['tweet']

In [43]:
def preprocess_data(documents):
    stop_words = stopwords.words("english")
    # Tokenize and remove stopwords
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in documents]
    return texts

In [44]:
processed_texts = preprocess_data(davidson_text)

In [45]:
# Create Dictionary
id2word = corpora.Dictionary(processed_texts)

In [46]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in processed_texts]

In [54]:
#Train the LDA model
# Set number of topics
num_topics = 5

# Build LDA model
lda_model = LdaModel(corpus=corpus, 
                     id2word=id2word, 
                     num_topics=num_topics, 
                     random_state=42, 
                     passes=10, 
                     alpha="auto", 
                     per_word_topics=True)


In [55]:
# Print the keywords for each topic
pprint(lda_model.print_topics())
## Maybe this simple text preprocessing is not enough for social media tweets? What do you think?
## Maybe k does not the have the optimal value
## Maybe training parameters need to be adjusted
## Maybe the models needs more data (LDA is an unsupervised model)

[(0,
  '0.069*"co" + 0.066*"http" + 0.045*"pussy" + 0.030*"rt" + 0.009*"bird" + '
  '0.007*"take" + 0.006*"eat" + 0.005*"yellow" + 0.005*"yankees" + '
  '0.005*"https"'),
 (1,
  '0.041*"trash" + 0.003*"racist" + 0.003*"place" + 0.003*"thirsty" + '
  '0.003*"white" + 0.003*"via" + 0.003*"fags" + 0.003*"literally" + '
  '0.003*"giving" + 0.003*"brownie"'),
 (2,
  '0.014*"stupid" + 0.011*"ghetto" + 0.011*"big" + 0.007*"another" + '
  '0.005*"wtf" + 0.004*"basic" + 0.004*"hot" + 0.004*"wait" + 0.004*"weed" + '
  '0.004*"niccas"'),
 (3,
  '0.066*"bitch" + 0.058*"rt" + 0.027*"bitches" + 0.023*"like" + 0.020*"hoes" '
  '+ 0.016*"hoe" + 0.012*"ass" + 0.011*"get" + 0.011*"shit" + 0.010*"nigga"'),
 (4,
  '0.015*"trash" + 0.013*"gt" + 0.011*"charlie" + 0.005*"bc" + 0.005*"throw" + '
  '0.005*"retard" + 0.005*"everybody" + 0.005*"hair" + 0.005*"years" + '
  '0.004*"dyke"')]


In [59]:
#Evaluate the model using the coherence score
#Higher coherence scores indicate a better model. You can try with a set of different k to choose the best model!
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_texts, dictionary=id2word, coherence="c_v")
coherence_lda = coherence_model_lda.get_coherence()
print("Coherence Score: ", coherence_lda)

/Users/ozgealacam/Library/CloudStorage/OneDrive-Personal/00_LMU/HateSpeech23_24/lecture_content/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ozgealacam/Library/CloudStorage/OneDrive-Personal/00_LMU/HateSpeech23_24/lecture_content/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ozgealacam/Library/CloudStorage/OneDrive-Personal/00_LMU/HateSpeech23_24/lecture_content/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/ur

Coherence Score:  0.4375253917831435
